In [1]:
%load_ext autoreload
%autoreload 2


In [66]:

import requests
import json
import socket
from typing import List, Dict


In [67]:
def test_if_sever_is_up():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    result = sock.connect_ex(('localhost', 80))
    if result == 0:
        print("Sever is up and running")
    else:
        print("Server is down. Please run `docker run --rm -p 80:80 reaction_wrapper`")

# Intro

* This notebook contains example usage of a `reaction_wrapper` service for running chemical reactions on provided reactans. 

* Make sure if server is up (run `test_if_sever_is_up()`).

* For testing  `get_reaction_products` was created, and it was run against for input jsons: one correct, and three with invalid inputs. Results are below.


In [76]:
test_if_sever_is_up()

Sever is up and running


In [77]:
def get_reaction_products(input_string: str, address: str = "localhost", port=80, limits=1000) -> Dict:
    """Function for processing input JSON

    Parameters
    ----------
    input_string : str
        _description_
    address : str, optional
        _description_, by default "localhost"
    port : int, optional
        _description_, by default 80
    limits : int, optional
        _description_, by default 1000

    Returns
    -------
    Dict
        response from the server
    """

    headers = {'Content-Type': 'application/json'}
    data = input_string

    response = requests.post(
        f'http://{address}:{port}/run_reaction?limits={limits}', headers=headers, data=data)

    return response.json()


In [84]:
input_string = '{"reaction_smarts": "[c:8]-[c:6]>>[c:8][I:55].[B:99][c:6]", "reactants": "CC1=CC=C(C=C1)C1=CC(=CC=C1C)C1=CC(C)=CC(C)=C1"}'
input_string_incorrect_smiles = '{"reaction_smarts": "[c:8]-[c:6]>>[c:8][I:55].[B:99][c:6]", "reactants": "ABCDEHGF"}'
input_string_incompatible_reactant = '{"reaction_smarts": "[c:8]-[c:6]>>[c:8][I:55].[B:99][c:6]", "reactants": "O=C(C)Oc1ccccc1C(=O)O"}'
input_string_incorrect_smarts = '{"reaction_smarts": "ABC", "reactants": "CC1=CC=C(C=C1)C1=CC(=CC=C1C)C1=CC(C)=CC(C)=C1"}'


## Correct input

In [82]:
get_reaction_products(input_string)

{'Products': [['Cc1cc(C)cc(-c2ccc(C)c(I)c2)c1', 'Bc1ccc(C)cc1'],
  ['Cc1ccc(-c2cc(I)ccc2C)cc1', 'Bc1cc(C)cc(C)c1'],
  ['Cc1cc(C)cc(I)c1', 'Bc1ccc(C)c(-c2ccc(C)cc2)c1'],
  ['Cc1ccc(I)cc1', 'Bc1cc(-c2cc(C)cc(C)c2)ccc1C']]}

In [83]:
get_reaction_products(input_string, limits = 1)

{'Products': [['Cc1ccc(I)cc1', 'Bc1cc(-c2cc(C)cc(C)c2)ccc1C']]}

## Incorrect inputs

In [80]:
get_reaction_products(input_string_incorrect_smiles)

{'message': 'Invalid reactant SMILES'}

In [81]:
get_reaction_products(input_string_incompatible_reactant)

{'message': 'Reactants not compatible with the reaction'}

In [85]:
get_reaction_products(input_string_incorrect_smarts)

{'message': 'Invalid reactant SMILES'}